# Function

In [48]:
h = 1e-4 # 스텝
dx = lambda f, x: (f(x+h) - f(x-h)) / (2*h) # 중앙 차분

class Function1:

    def __init__(self):
        self.function = lambda x: x - 2
    
    def forward(self, x):
        self.x = x
        z = self.function(x)
        return z
    
    def backward(self, dy_dx):
        self.dx = dx(f=self.function, x=self.x)
        self.dout = self.dx * dy_dx
        return self.dout

class Function2:
    
    def __init__(self):
        self.function = lambda z: 2 * z**2

    def forward(self, z):
        self.z = z
        y = self.function(z)
        return y
    
    def backward(self):
        self.dx = dx(f=self.function, x=self.z)
        self.dout = self.dx
        return self.dout
        
class Function:
    
    def __init__(self):
        self.function_1 = Function1()
        self.function_2 = Function2()
    
    def forward(self, x):
        print(f"Input x = {x}")
        z = self.function_1.forward(x)
        print(f"z = {z}")
        y = self.function_2.forward(z)
        print(f"y = {y}")
        return y

    def backward(self):
        f2_dout = self.function_2.backward()
        f1_dout = self.function_1.backward(f2_dout)
        print(f"Function2 dx = {self.function_2.dx}")
        print(f"Function1 dx = {self.function_1.dx}")
        print(f"Chain Rule dx = {f1_dout}")
        return f1_dout

function = Function()
function.forward(x=5)
function.backward()

# 전방 차분 결과 값
# Function2 dx = 12.000200000024108
# Function1 dx = 0.9999999999976694
# Chain Rule dx = 12.000199999996141

Input x = 5
z = 3
y = 18
Function2 dx = 12.000000000025324
Function1 dx = 0.9999999999976694
Chain Rule dx = 11.999999999997357


11.999999999997357